In [2]:
''' 
packages
'''


import pandas as pd 
import ast
from googletrans import Translator
from multiprocessing import Pool, Manager
import numpy as np
from collections import Counter




In [ ]:

''' 

Translation of transcibing file

'''


df_transciption = pd.read_csv('data/complete_data_transcribing.csv')
df_transciption = df_transciption[df_transciption.sentences != 'failed']
df_transciption['ids'] = df_transciption['id'] 
df_transciption['comp'] = np.arange(len(df_transciption))
del df_transciption['Unnamed: 0']
translator = Translator()
def update_data_with_trans(A):
    d = A[0]
    i = A[1]
    extracted =  df_transciption[df_transciption.comp == i]['sentences'].values[0]
    d[i] = translator.translate(extracted, dest='en').text

manager = Manager()
translated_dict = manager.dict()
with Pool(15) as p:
    p.map(update_data_with_trans, [[translated_dict,i] for i in np.arange(len(df_transciption))])
df_transciption['translated'] = [translated_dict[i] for i in np.arange(len(df_transciption))]
del df_transciption['comp']
del df_transciption['ids']
df_transciption.to_csv('data/complete_data_transcribing_translated.csv', index=False)


In [ ]:
'''
Translation of attributes file 

'''



df_attributes = pd.read_csv('data/videos_alphapolitica_complete_info_ids.csv')
lis_tag = [ast.literal_eval(i) for i in df_attributes['tags']]
lis_tags = [j for i in lis_tag for j in i]
dict_counter = dict(Counter(lis_tags))
df_counter = pd.DataFrame()
df_counter['tags'] = dict_counter.keys()
df_counter['counter'] = dict_counter.values()
df_counter = df_counter.sort_values(by='counter', ascending= False)
stopwords = [i.lower() for i in df_counter[df_counter.counter > 7000]['tags'].values]
to_drop = ['today ap news', 'today telugu news','tv5 telugu live','ap news live','ap political news']
stopwords = stopwords + to_drop
df_attributes['tags'] = [ast.literal_eval(i) for i in df_attributes['tags']]
df_attributes['tags_updated'] = [list(set([j.lower() for j in i ])-set(stopwords)) for i in  df_attributes['tags'].values]
df_attributes['tags'] = [' '.join(i) for i in df_attributes['tags_updated']]
del df_attributes['tags_updated']
del df_attributes['Unnamed: 0']
df_attributes = df_attributes.dropna(subset=['title'])
df_attributes['tags'] = [' '.join(ast.literal_eval(i)) for i in df_attributes['tags']]
df_attributes = df_attributes[['ids','title','tags','upload_date','duration','comment_count','view_count','uploader']]
df_attributes['comp'] = np.arange(len(df_attributes))
translator = Translator()
def update_data_with_trans(A):
    d = A[0]
    i = A[1]
    extracted =  df_attributes[df_attributes.comp == i]['title'].values[0]
    d[i] = translator.translate(extracted, dest='en').text

manager = Manager()
translated_dict = manager.dict()
with Pool(15) as p:
    p.map(update_data_with_trans, [[translated_dict,i] for i in np.arange(len(df_attributes))])
df_attributes['title_translated'] = [translated_dict[i] for i in np.arange(len(df_attributes))]
del df_attributes['comp']
df_attributes.to_csv('data/videos_alphapolitica_complete_info_ids_translated.csv', index= False)


In [4]:
df = pd.read_csv('data/videos_alphapolitica_complete_info_ids_translated_update.csv')
df_transciption = pd.read_csv('data/complete_data_transcribing_translated.csv')
df_attributes = pd.read_csv('data/videos_alphapolitica_complete_info_ids_translated_update.csv')
df_attributes_merged = df_attributes.merge(df_transciption, on='ids', how='outer')
df_attributes_merged = df_attributes_merged.dropna(subset = ['title'])
df_attributes_merged = df_attributes_merged[['ids','uploader','translated','tags_translated','title_translated','upload_date','duration','comment_count','view_count']]
df_attributes_merged.to_csv('data/data_compiled_final_may_2023.csv', index=False)

In [9]:
''' 

Data is created, paraphrasing ! 

'''

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sys
from tqdm import tqdm
import pandas as pd
import numpy as np

/Users/kowshik/projects/personal/alphapolitica/alphapenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:


device = "cuda:0"
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=1280
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    input_ids = input_ids.to(device)
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res

df_data  = pd.read_csv('data_compiled_final_may_2023.csv')
df_data['translated'] =  df_data['translated'].fillna('')

translated_paraphrase = []
for i in tqdm(range(len(df_data))):
    text = df_data['translated'].values[i]
    if text == '':
        translated_paraphrase.append(text)
    else:
        translated_paraphrase.append(paraphrase(text)[0])

df_data['translated_paraphrase'] = translated_paraphrase
df_data.to_csv('data_compiled_final_may_2023_paraphrased.csv', index = False)


In [ ]:
''' 

Classification

'''

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sys
from tqdm import tqdm
import pandas as pd
import numpy as np

/Users/kowshik/projects/personal/alphapolitica/alphapenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_data  = pd.read_csv('data/data_compiled_final_may_2023.csv')
df_data['translated'] =  df_data['translated'].fillna('')
df_data['tags_translated'] =  df_data['tags_translated'].fillna('')
df_data['title_translated'] =  df_data['title_translated'].fillna('')
df_data['sentence_for_classification'] = df_data['title_translated'] + ' ' + df_data['translated'] + ' ' +  df_data['tags_translated'] 



In [8]:
df_data['sentence_for_classification'].values[25]

"Shouldn't YCP be ashamed for being the reason why Chandrababu said that? | Last Chance | The Debate  "

In [ ]:
topics = ['chandhra babu','YCP', 'jagan', 'TDP','murder',
 'vivek', 'roads', 'capital', 'atrocities','volunteer', 'amma vadi',
  'welfare', 'schemes', 'lokesh', 'pattabi', 'vijayawada', 'visaka',
   'jogi ramesh','vamsi','kodali','nani', 'Education','Agriculture',
   'farmers','Corruption','investment','Budget','SC/ST','Polavaram',
   'irrigation','YSRCP','Telugu Desam Party','Vidya Deevena','education',
   'governance', ]


In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device = 'cuda:0')


In [ ]:

topics = ['chandhra babu','YCP', 'jagan', 'TDP','murder', 'vivek', 'roads', 'capital', 'atrocities','volunteer', 'amma vadi', 'welfare', 'schemes', 'lokesh', 'pattabi', 'vijayawada', 'visaka', 'jogi ramesh','vamsi','kodali','nani']
predictions = {}
for topic in topics:
    predictions[topic] = []
    sentences = df_one_shot['to_pred'].values
    for sent in tqdm(sentences):
        out = classifier(sent, ['chandhra babu'])
        predictions[topic].append(out['scores'][0])

In [ ]:
sequence_to_classify

In [ ]:
df_para['para_phrased'].values[0]